In [303]:
import numpy as np
import plotly.graph_objects as go
from ipywidgets import widgets
import ipywidgets
from IPython.display import display, Math, Latex

In [304]:
#Création des curseurs de sélection des paramètres du système

updt=False #variable de mise à jour continue
dsb=False #variable de disponibilité du widget

style = {'description_width': 'initial','button_width':'initial'} #style permettant de gérer l'affichage complet du texte
maxi=[60.0,50.0,14,100,40,60,60,4000,100]#tableau des maximaux initiaux des sliders 
#FloatSliders pour les paramètres
Fiso= widgets.FloatSlider(
    value=45.0,
    min=0.1,
    max=maxi[0],
    step=0.001,
    description='$F_{iso}$',
    continuous_update=updt,
    disabled=dsb,
    style=style,
    tooltip='Description',
    )
Rfb= widgets.FloatSlider(
    value=3,
    min=0.1,
    max=maxi[1],
    step=0.001,
    description="$R_{fb}$",
    continuous_update=updt,
    disabled=dsb,
    style=style
)
It= widgets.FloatSlider(
    value=10.0,
    min=0.1,
    max=maxi[2],
    step=0.001,
    description="$I_T$",
    continuous_update=updt,
    disabled=dsb,
    style=style
)
Rm= widgets.FloatSlider(
    value=39,
    min=0.1,
    max=maxi[4],
    step=0.001,
    description="$R_M$",
    continuous_update=updt,
    disabled=dsb,
    style=style
)
B= widgets.FloatSlider(
    value=16.0,
    min=0.1,
    max=maxi[3],
    step=0.001,
    description="$B$",
    continuous_update=updt,
    disabled=dsb,
    style=style
)


Mu=widgets.FloatRangeSlider(
    value=[10.0,40.0],
    min=0.1,
    max=maxi[7],
    step=0.001,
    description='$\mu_--\mu_+$',
    continuous_update=updt,
    disabled=not dsb,
    color='#C14535',
    style=style
    )
Rp= widgets.FloatSlider(
    value=45.0,
    min=0.1,
    max=maxi[6],
    step=0.001,
    description='$R_+$',
    continuous_update=updt,
    disabled=not dsb,
    style=style
    )
Re= widgets.FloatSlider(
    value=45.0,
    min=0.1,
    max=maxi[5],
    step=0.001,
    description='$R_E$',
    continuous_update=updt,
    disabled=not dsb,
    style=style
    )
Alph= widgets.FloatSlider(
    value=45.0,
    min=0.1,
    max=maxi[8],
    step=0.001,
    description='$\propto $',
    continuous_update=updt,
    disabled=not dsb,
    style=style
    )
#Choix multiple pour le modèle de corps à suivre
Corps=widgets.RadioButtons(
    options=['sportif', 'sédentaire', 'obèse','aucun'],
    value='aucun',
    layout={'width': 'max-content'},
    description='Modèle suivi',
    disabled=False
)

In [305]:
#Bouton pour l'évolution des courbes en temps réel
BoutonMaj=widgets.Checkbox(
    value=False,
    description='Mise à jour continue',
    disabled=False,
    button_style='', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Description',
    style=style
)

def temps_reel(updt):
    Fiso.continuous_update=updt
    Rfb.continuous_update=updt
    It.continuous_update=updt
    Rm.continuous_update=updt
    B.continuous_update=updt
    Re.continuous_update=updt
    Rp.continuous_update=updt
    Alph.continuous_update=updt
    Mu.continuous_update=updt
temps_reel(updt)


In [306]:
#coefficients des paramètres
prop=[1,1/10,1/10,1/100,1,1,1,1,1,1/1000]

#inverse des coefficients
iprop=[1/x for x in prop]

#Fonction donnant les valeurs des paramètres
def val():
    return Fiso.value*prop[0],Rfb.value*prop[1],It.value*prop[2],Rm.value*prop[3],B.value*prop[4],Re.value*prop[5],Rp.value*prop[6],Mu.value[1]*prop[7],Mu.value[0]*prop[8],Alph.value*prop[9]

#Bouton de passage à la représentation duale
desc=['eprésentation métabolique','eprésentation thermodynamique']
BoutonDual=widgets.ToggleButton(
    value=False,
    description='R'+desc[0],
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Cliquer pour passer à la représentation thermodynamique',
    style=style,
    layout=widgets.Layout(width='auto')
)

#Calcul du facteur de mérite
def fm():
    F,R_fb,I_t,R_m,B_,R_e,R_p,Mu_p,Mu_m,A=val()
    return F*R_fb*I_t/(R_m*B_)

#Affichage du facteur de mérite
FM=widgets.HTMLMath(
    value=r"$f_M=\frac{R_{fb}F_{iso}I_T}{R_MB}=$ ",
    placeholder='',
    description='',
    tooltip='Description',
    style=style,
    layout=widgets.Layout(width='auto')
)
FM2=widgets.FloatText(
    value=(fm()//0.01)*0.01,
    description='',
    disabled=True,
    layout=widgets.Layout(width='auto')
)

In [307]:
#Fonction calculant l'étendue de l'axe des x
def fin():
    F,R_fb,I_t,R_m,B_,R_e,R_p,Mu_p,Mu_m,A=val()
    return((-(R_m+R_fb)*I_t+np.sqrt(((R_m+R_fb)**2)*(I_t**2)+4*F*R_m*I_t))/(2*R_m))
X=np.arange(0.0000,fin()+0.001,0.1)

#Fonction décrivant les flux d'énergie, la puissance et le COT suivant la vitesse
def f(x) :
    F,R_fb,I_t,R_m,B_,R_e,R_p,Mu_p,Mu_m,A=val()
    r=B_/(F*I_t)
    Eta=F/(F+R_fb*I_t*(1-r))
    #Eta=(Mu_p-Mu_m)/Mu_p+0.000001
    Rh=(F+R_fb*I_t)/(I_t+x)
    fp=(F*x/Eta+B_)*I_t/(I_t+x)
    fm=R_m*x*x+(R_fb*x*x+F*(1/Eta-1)*x+B_)*I_t/(I_t+x)
    p=(F-(R_m+Rh)*x)*x
    cot=fm/(x+0.000000000000000000000000000000000000001)
    rendement=p/fp
    force=F-(R_m+Rh)*x
    
    return [fp,fm,p,cot,rendement,force]
Y=f(X)

In [308]:
#Création des graphiquesfm

#Graphique de Puissance
trace1=go.Scatter(x=X,y=Y[0],name="Flux entrant")
trace2=go.Scatter(x=X,y=Y[1],name="Flux sortant")
trace3=go.Scatter(x=X,y=Y[2],name="Puissance")
g = go.FigureWidget(data=[trace1,trace2,trace3],
                    layout=go.Layout(
                        title=dict(text="Flux d'énergie et Puissance"),
                        paper_bgcolor="white",
                        plot_bgcolor="azure"
                    ))
g.update_xaxes(title="v en m/s")
g.update_yaxes(title="puissance (unités arbitraires)")

#Police et forme
g.update_layout(
    font=dict(
            family="sans-serif",
            size=15,
            color="black"
    ),
    legend=dict(
        x=0,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    )
)

#Graphique de COT
trace4=go.Scatter(x=X,y=Y[3],name="COT")
g2 = go.FigureWidget(data=[trace4],layout=g.layout)
g2.update_layout(title=dict(text="COT"),legend=dict(x=-2,y=1))
g2.update_xaxes(title="v en m/s")
g2.update_yaxes(title="COT (unités arbitraires)")

#Graphique Rendement/Puissance
trace5=go.Scatter(x=Y[2],y=Y[4],name="Rendement",line=dict(color='green'))
g4 = go.FigureWidget(data=[trace5],layout=g.layout)
g4.update_layout(title=dict(text="Rendement/Puissance"),legend=dict(x=-2,y=1))
g4.update_xaxes(title="Puissance (unités arbitraires)")
g4.update_yaxes(title="Rendement (unités arbitraires)")


#Graphique Force
trace6=go.Scatter(x=X[0:len(X)-1],y=Y[5][2:len(X)-1],name="Force",line=dict(color='black'))
g5 = go.FigureWidget(data=[trace6],layout=g.layout)
g5.update_layout(title=dict(text="Force"),legend=dict(x=-2,y=1))
g5.update_xaxes(title="v en m/s")
g5.update_yaxes(title="Force (unités arbitraires)")

#Graphique polaire
trace7=go.Scatterpolar(
    name = "Paramètres actuels",
    theta = ["<b>Force isométrique<b>", "<b>Résistance de rétroaction<b>", "<b>Seuil d'intensité<b>", "<b>Flux basal<b>", "<b>Résistance de dissipation<b>"],
    r = 5*np.array([Fiso.value/Fiso.max, Rfb.value/Rfb.max, It.value/It.max, B.value/B.max, Rm.value/Rm.max]),
    fill='toself'
    )
trace8=go.Scatterpolar(name="Sportif",r=[4,3,4,4,2],theta = ["<b>Force isométrique<b>", "<b>Résistance de rétroaction<b>", "<b>Seuil d'intensité<b>", "<b>Flux basal<b>", "<b>Résistance de dissipation<b>"],fill='toself')
trace9=go.Scatterpolar(name="Sédentaire",r=[3.001,2.001,3.001,3.001,3.001],theta = ["<b>Force isométrique<b>", "<b>Résistance de rétroaction<b>", "<b>Seuil d'intensité<b>", "<b>Flux basal<b>", "<b>Résistance de dissipation<b>"],fill='toself')
trace10=go.Scatterpolar(name="Obèse",r=[3.002,2.0002,1.002,4.998,1.002],theta = ["<b>Force isométrique<b>", "<b>Résistance de rétroaction<b>", "<b>Seuil d'intensité<b>", "<b>Flux basal<b>", "<b>Résistance de dissipation<b>"],fill='toself')

g3 = go.FigureWidget(data=[trace7,trace8,trace9,trace10],layout=g.layout)
g3.update_layout(title=dict(text="Paramètres"),legend=dict(x=1,y=1))

#Nom des axes

#Annotations
def abx():#Pour obtenir les coordonnées des annotations
    pmax=np.amax(g.data[2].y)
    ixpmax=np.where(g.data[2].y==pmax)[0][0]
    xpmax=g.data[2].x[ixpmax]
    cotmin=np.amin(g2.data[0].y)
    ixcotmin=np.where(g2.data[0].y==cotmin)[0][0]
    xcotmin=g2.data[0].x[ixcotmin]
    rendmax=np.amax(g4.data[0].y)
    iprendmax=np.where(g4.data[0].y==rendmax)[0][0]
    prendmax=g4.data[0].x[iprendmax]
    return (pmax,ixpmax,xpmax,cotmin,ixcotmin,xcotmin,rendmax,prendmax)
    
def annotations1(g):
    pmax,ixpmax,xpmax,cotmin,ixcotmin,xcotmin,rendmax,prendmax=abx()   
    g.update_layout( annotations=[
        dict(
            x=0,
            y=g.data[0].y[0],
            xref="x",
            yref="y",
            text="Flux basal",
            showarrow=True,
            arrowhead=7,
            ax=80,
            ay=-40,
            #font=dict(color="red")
        ),
        dict(
            y=pmax,
            x=xpmax,
            xref="x",
            yref="y",
            text="Maximum de puissance",
            showarrow=True,
            arrowhead=7,
            ax=80,
            ay=-40
        ),
        dict(
            y=0,
            x=fin(),
            xref="x",
            yref="y",
            text="Épuisement",
            showarrow=True,
            arrowhead=7,
            ax=0,
            ay=-40
        )
    ])
annotations1(g)

def annotations2(g):
    pmax,ixpmax,xpmax,cotmin,ixcotmin,xcotmin,rendmax,prendmax=abx()
    if g2.data[0].y[-1]!=cotmin :
        g.update_layout( annotations=
            [dict(
                x=xcotmin,
                y=cotmin,
                xref="x",
                yref="y",
                text="COT*",
                showarrow=True,
                arrowhead=7,
                ax=10,
                ay=-40,
                #font=dict(color="red")
            )])
annotations2(g2)

def annotations3(g):
    pmax,ixpmax,xpmax,cotmin,ixcotmin,xcotmin,rendmax,prendmax=abx()
    pcotmin=f(xcotmin)[2]
    g4.update_layout( annotations =
         [dict(
                x=pmax,
                y=g4.data[0].y[ixpmax],
                xref="x",
                yref="y",
                text="Puissance max",
                showarrow=True,
                arrowhead=7,
                ax=10,
                ay=-50,
                #font=dict(color="red")
            ),
         dict(
                x=pcotmin,
                y=g4.data[0].y[ixcotmin],
                xref="x",
                yref="y",
                text="COT*",
                showarrow=True,
                arrowhead=7,
                ax=10,
                ay=40,
                #font=dict(color="red")
            ),
         dict(
                x=prendmax,
                y=rendmax,
                xref="x",
                yref="y",
                text="Rendement max",
                showarrow=True,
                arrowhead=7,
                ax=-50,
                ay=-20,
                #font=dict(color="red")
            )])
annotations3(g4)    


#Fonction d'auto-ajustement de l'échelle du graphique de COT
def echelle(g2):
    cotf=g2.data[0].y[-1]
    cotmin=np.amin(g2.data[0].y)
    cotb=cotmin*0.90
    coth=cotf+2*(cotf-cotb)
    g2.update_yaxes(range=[cotb,coth])
echelle(g2)    

#Mise à jour des paramètres avec leurs coefficients
def maj(F,R_fb,I_t,R_m,B_,R_e,R_p,Mu_p,Mu_m,A):   
    Fiso.max=max(F*iprop[0],maxi[0])
    Rfb.max=max(R_fb*iprop[1],maxi[1])
    It.max=max(I_t*iprop[2],maxi[2])
    Rm.max=max(R_m*iprop[3],maxi[3])
    B.max=max(B_*iprop[4],maxi[4])
    Re.max=max(R_e*iprop[5],maxi[5])
    Rp.max=max(R_p*iprop[6],maxi[6])
    Mu.max=max(Mu_p*iprop[8],maxi[7])
    Alph.max=max(A*iprop[9],maxi[8])
    Fiso.value,Rfb.value,It.value,Rm.value,B.value,Re.value,Rp.value,Mu.value,Alph.value=F*iprop[0],R_fb*iprop[1],I_t*iprop[2],R_m*iprop[3],B_*iprop[4],R_e*iprop[5],R_p*iprop[6],[Mu_m*iprop[8],Mu_p*iprop[7]],A*iprop[9]
    
#Réponse des graphiques aux changements de paramètres
def response():
    X=np.arange(0.0000,fin()+0.1,0.1)
    Y=f(X)
    with g.batch_update():
        g.data[0].x = X
        g.data[1].x = X
        g.data[2].x = X
        g.data[0].y = Y[0]
        g.data[1].y = Y[1]
        g.data[2].y = Y[2]
    with g2.batch_update():
        g2.data[0].x=X
        g2.data[0].y=Y[3]
    with g2.batch_update():
        echelle(g2)
        annotations2(g2)
    with g.batch_update():
        annotations1(g)
    with g4.batch_update():
        g4.data[0].x=Y[2]
        g4.data[0].y=Y[4]
    with g4.batch_update():
        annotations3(g3)
    with g5.batch_update():
        g5.data[0].x=X
        g5.data[0].y=Y[5]
    with g3.batch_update():
        g3.data[0].r=5*np.array([Fiso.value/maxi[0], Rfb.value/maxi[1], It.value/maxi[2], B.value/maxi[4], Rm.value/maxi[3] ])
    if c[1]==True : Corps.value='aucun'
    c[1]=True



In [309]:
#Mise à jour des graphiques 

#Mise à jour des paramètres en fonction du modèle choisi
def majmod(C):
    Fiso.value=C[0]*maxi[0]/5
    Rfb.value=C[1]*maxi[1]/5
    It.value=C[2]*maxi[2]/5
    B.value=C[3]*maxi[4]/5
    Rm.value=C[4]*maxi[3]/5
    
#Booléen de Mise à jour
c=[4,True]

#Mise à jour des paramètres thermo en fonction des paramètres métaboliques
def majth():
    F,R_fb,I_t,R_m,B_,R_e,R_p,Mu_p,Mu_m,A=val()
    r=min(B_/(F*I_t),1-10**(-2))
    R_p=R_e*r/(1-r)
    A=F/(R_e*B_)
    Mu_m=R_fb*I_t/A
    Mu_p=Mu_m*(F+R_fb*(1-r)*I_t)/(R_fb*(1-r)*I_t)
    maj(F,R_fb,I_t,R_m,B_,R_e,R_p,Mu_p,Mu_m,A)
    if c[0]==4 : response()
    else : c[0]+=1
majth()

def response2(change):
    FM2.value=(fm()//0.01)*0.01
    b=BoutonDual.value
    if not b or not c[1]:
        majth()

#Mise à jour des paramètres métaboliques en fonction des paramètres thermo
def majmet():
    F,R_fb,I_t,R_m,B_,R_e,R_p,Mu_p,Mu_m,A=val()
    r=R_p/(R_p+R_e)
    I_t=1/(A*r*R_e)
    R_fb=A*Mu_m/I_t
    F=A*(Mu_p-Mu_m)*(1-r)
    B_=F/(A*R_e)
    maj(F,R_fb,I_t,R_m,B_,R_e,R_p,Mu_p,Mu_m,A)
    response()
majmet()

def response3(change):
    b=BoutonDual.value
    if b and c[1]:
        majmet()

#Mise à jour des paramètres en fonction du modèle choisi 
#La fonction dec triche et modifie infinitésimalement les paramètres quand on bascule sur 'aucun' 
def dec():
    C=[]
    for x in g3.data[0].r :
        C.append(x+0.00000000000001)
    return C
        
def responseCorps(change):
    c[0]=0
    c[1]=False
    if Corps.value=='sportif': majmod(g3.data[1].r)
    elif Corps.value=='sédentaire':majmod(g3.data[2].r)
    elif Corps.value=='obèse':majmod(g3.data[3].r)
    elif Corps.value=='aucun':majmod(dec())    

#Mise à jour du set de paramètres servant à la représentation
def responseDual(change):
    dsb = BoutonDual.value
    BoutonDual.description='R'+desc[BoutonDual.value]
    BoutonDual.tooltip='Cliquer pour passer à la r'+desc[not BoutonDual.value]
    Mu.disabled,Alph.disabled,Rp.disabled,Re.disabled= not dsb,not dsb,not dsb,not dsb
    Fiso.disabled,Rfb.disabled,B.disabled,Rm.disabled,It.disabled=dsb, dsb, dsb, dsb, dsb

#Mise à jour de la continuité de la mise à jour des graphiques
def responseMaj(change):
    temps_reel(BoutonMaj.value)

#Observation des changements
Fiso.observe(response2, names="value")
Rfb.observe(response2, names="value")
It.observe(response2, names="value")
B.observe(response2, names="value")
Rm.observe(response2, names="value")
Rp.observe(response3, names="value")
Re.observe(response3, names="value")
Mu.observe(response3, names="value")
Alph.observe(response3, names="value")
BoutonMaj.observe(responseMaj, names="value")
Corps.observe(responseCorps, names="value")
BoutonDual.observe(responseDual, names="value")

In [310]:
box_layout = widgets.Layout(width='1500px',min_width='1000px',max_width='2000px')
box_layout2 = widgets.Layout(heigth='0px')
#print(widgets.Layout.main-end.__doc__)
#Affichage des éléments
Meta=widgets.HTML(value="<b>Paramètres métaboliques<b>")
Thermo=widgets.HTML(value="<b>Paramètres thermodynamiques<b>")
container1=widgets.HBox([Fiso,Rfb,BoutonMaj])
container2=widgets.HBox([It,B,BoutonDual])
container3=widgets.HBox([Rm,FM,FM2])
bc=widgets.VBox([container1,container2,container3])
bcv=widgets.HBox([bc,Corps])
bch=widgets.VBox([Meta,bcv])
bc2=widgets.VBox([Thermo,Mu,Rp,Re,Alph])
bbc=widgets.HBox([bch,bc2])
container6=widgets.HBox([g,g3])
w=widgets.VBox([bbc,container6],box_style='')
ww=widgets.Box(children=[g2,g4,g5],layout=box_layout)
display(w)
display(ww)
#maj(45.0,3,8.0,39,16.0,45.0,45.0,10.0,40.0,45.0)
#maj(45.0,3,10.0,39,16.0,45.0,45.0,10.0,40.0,45.0)
display(widgets.Box(children=[],layout=box_layout2))

Box(children=(FigureWidget({
    'data': [{'name': 'COT',
              'type': 'scatter',
              'uid'…

Box()